<a href="https://colab.research.google.com/github/agalvezm/ACE2_scRNAseq/blob/master/scvi/scvi_ldvae_concatenated_anndatas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import and install packages

In [5]:
!pip install scikit-misc
!pip install pillow scikit-learn

# Install scvi
!pip install scvi-tools
#!pip install scvi-tools==0.8.0b0


!pip install scanpy anndata

!pip install scanpy python-igraph louvain MulticoreTSNE pybiomart

#import packages
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import anndata
import scvi


# Import packages
import anndata
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
from sklearn.decomposition import TruncatedSVD
from scipy import sparse, io

matplotlib.rcParams.update({'font.size': 12})
%config InlineBackend.figure_format = 'retina'

def nd(arr):
    return np.asarray(arr).reshape(-1)

     |████████████████████████████████| 8.7MB 4.6MB/s 
     |████████████████████████████████| 153kB 5.4MB/s 
     |████████████████████████████████| 245kB 36.6MB/s 
     |████████████████████████████████| 122kB 64.4MB/s 
     |████████████████████████████████| 194kB 74.4MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
  Found existing installation: openpyxl 2.5.9
    Uninstalling openpyxl-2.5.9:
      Successfully uninstalled openpyxl-2.5.9
  Found existing installation: importlib-metadata 3.4.0
    Uninstalling importlib-metadata-3.4.0:
      Successfully uninstalled importlib-metadata-3.4.0
     |████████████████████████████████| 10.3MB 5.3MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
  Created wheel for sinfo: filename=sinfo-0.3.1-cp36-none-any.whl size=7014 sha256=a4d8b668551838f3e322e70c6d846c2e03c172e016b9224220f5418422891671
  Stored in directory: /root/.cache/pip/wheels/11/f0/23/347d6d8e59787c2bc27

In [1]:
!pip3 install torch torchvision

## Download concatenated anndata

In [3]:
# No hvg filters, all data
!wget -O concatenated_adata.dms 

--2021-02-03 21:28:58--  https://caltech.box.com/shared/static/3izsxihbu8vo9k2tf44et42mlpnwm3h5
Resolving caltech.box.com (caltech.box.com)... 103.116.4.197
Connecting to caltech.box.com (caltech.box.com)|103.116.4.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/3izsxihbu8vo9k2tf44et42mlpnwm3h5 [following]
--2021-02-03 21:28:58--  https://caltech.box.com/public/static/3izsxihbu8vo9k2tf44et42mlpnwm3h5
Reusing existing connection to caltech.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://caltech.app.box.com/public/static/3izsxihbu8vo9k2tf44et42mlpnwm3h5 [following]
--2021-02-03 21:28:58--  https://caltech.app.box.com/public/static/3izsxihbu8vo9k2tf44et42mlpnwm3h5
Resolving caltech.app.box.com (caltech.app.box.com)... 103.116.4.201
Connecting to caltech.app.box.com (caltech.app.box.com)|103.116.4.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https

## Read and filter anndata object

In [7]:
adata = scvi.data.read_h5ad("concatenated_adata.dms")


In [8]:
sc.pp.highly_variable_genes(adata, flavor='seurat_v3', n_top_genes=3000) # subset=True)

/usr/local/lib/python3.6/dist-packages/scanpy/preprocessing/_highly_variable_genes.py:144: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  df.loc[: int(n_top_genes), 'highly_variable'] = True


In [10]:
adata

AnnData object with n_obs × n_vars = 1165808 × 39442
    obs: 'cell_counts', 'n_genes', 'percent_mito', 'pass_count_filter', 'pass_mito_filter', 'cell_type', 'condition', 'database_id', 'figure', 'paper', 'sample_id', 'species', 'technology', 'tissue'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg'

In [11]:
#Remove empty cells
sc.pp.filter_cells(adata, min_genes = 1)

In [ ]:
adata

AnnData object with n_obs × n_vars = 1165694 × 39442
    obs: 'cell_counts', 'n_genes', 'percent_mito', 'pass_count_filter', 'pass_mito_filter', 'cell_type', 'condition', 'database_id', 'figure', 'paper', 'sample_id', 'species', 'technology', 'tissue'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg'

In [ ]:
adata.write("adata_concatenated_filtered_post.dms")

In [12]:
scvi.data.setup_anndata(adata, batch_key="sample_id") # , layer="counts")

INFO     Using batches from adata.obs["sample_id"]                              
INFO     No label_key inputted, assuming all cells have same label              
INFO     Using data from adata.X                                                
INFO     Computing library size prior per batch                                 
INFO     Successfully registered anndata object containing 1165694 cells, 39442 
         vars, 195 batches, 1 labels, and 0 proteins. Also registered 0 extra   
         categorical covariates and 0 extra continuous covariates.              
INFO     Please do not further modify adata until model is trained.             


In [13]:
scvi.data.view_anndata_setup(adata)

Anndata setup with scvi-tools version 0.8.1.


In [14]:
# initialize LinearSCVI model
model = scvi.model.LinearSCVI(adata, n_latent=10)

In [15]:
# train for 250 epochs, compute metrics every 10 epochs
model.train(n_epochs=250, lr = 5e-3, frequency = 10)

INFO     Training for 250 epochs                                                
INFO     KL warmup phase exceeds overall training phaseIf your applications rely
         on the posterior quality, consider training for more epochs or reducing
         the kl warmup.                                                         
INFO     KL warmup for 400 epochs                                               
Training...: 100%|██████████| 250/250 [16:07:14<00:00, 232.14s/it]
INFO     Training is still in warming up phase. If your applications rely on the
         posterior quality, consider training for more epochs or reducing the kl
         warmup.                                                                
INFO     Training time:  54523 s. / 250 epochs                                  


In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

#ignore plotting elbo of first epoch
train_elbo = model.trainer.history['elbo_train_set'][1:]
test_elbo = model.trainer.history['elbo_test_set'][1:]
epochs = np.arange(10,260,10)

ax.plot(epochs, train_elbo, label='train')
ax.plot(epochs, test_elbo, label='test')
ax.legend()

In [ ]:
Z_hat = model.get_latent_representation()
for i, z in enumerate(Z_hat.T):
    adata.obs[f'Z_{i}'] = z

In [ ]:
fig = plt.figure(figsize=(12, 8))

for f in range(0, 9, 2):
    plt.subplot(2, 3, int(f / 2) + 1)

    plt.scatter(adata.obs[f'Z_{f}'], adata.obs[f'Z_{f + 1}'], marker='.', s=4, label='Cells')

    plt.xlabel(f'Z_{f}')
    plt.ylabel(f'Z_{f + 1}')

plt.subplot(2, 3, 6)
plt.scatter(adata.obs[f'Z_{f}'], adata.obs[f'Z_{f + 1}'], marker='.', label='Cells', s=4)
plt.scatter(adata.obs[f'Z_{f}'], adata.obs[f'Z_{f + 1}'], c='w', label=None)
plt.gca().set_frame_on(False)
plt.gca().axis('off')

lgd = plt.legend(scatterpoints=3, loc='upper left')
for handle in lgd.legendHandles:
    handle.set_sizes([200])


plt.tight_layout()